In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
from sys import api_version


api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
api_version = "2023-05-15"
api_base = "https://gpt-4o-fr.openai.azure.com"
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
    api_version=api_version,
    api_base=api_base
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../output/usage_ai/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 106
Report count after filtering by community level 2: 80


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,100,# Random Forest and AI Power Users\n\nThe comm...,2,8.5,Random Forest and AI Power Users,The impact rating is high due to the significa...,The community revolves around the Random Fores...,[{'explanation': 'The Random Forest model is t...,"{\n ""title"": ""Random Forest and AI Power Us...",3e0fa18f-216a-4095-a1d4-1bce2299771a
1,101,# Predictive Models and Model Performance in A...,2,9.0,Predictive Models and Model Performance in AI,The rating is high due to the significant impa...,The community is centered around the use of pr...,[{'explanation': 'Predictive models are centra...,"{\n ""title"": ""Predictive Models and Model P...",33c467c6-d236-4f62-b28b-9b8b5ac948dc
2,102,# Second Microsoft Report on AI and Productivi...,2,9.0,Second Microsoft Report on AI and Productivity...,The report is highly significant as it provide...,The community centers around the Second Micros...,[{'explanation': 'The Second Microsoft Report ...,"{\n ""title"": ""Second Microsoft Report on AI...",8d54fa21-ec89-46da-b4ae-8ce2de2f885e
3,103,# AI and Productivity Research Community: Key ...,2,9.0,AI and Productivity Research Community: Key Co...,The rating is high due to the significant impa...,The community centers around the research cont...,[{'explanation': 'Brian Houck is a contributin...,"{\n ""title"": ""AI and Productivity Research ...",360b2cc1-d0f8-456e-b0bf-aed193e8abd8
4,104,# Impact of Copilot for Security on Workplace ...,2,9.0,Impact of Copilot for Security on Workplace Pr...,The rating is high due to the comprehensive re...,The community centers around the research and ...,"[{'explanation': 'Ben Edelman, James Bono, Rob...","{\n ""title"": ""Impact of Copilot for Securit...",824c64e1-ee24-4de9-9a7a-25e267092faf


#### Build global context based on community reports

In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [8]:
result = await search_engine.asearch(
    "What is the major insights to remember ?"
)

print(result.response)

### Major Insights on the Impact of Generative AI Tools like Copilot

#### Enhanced Productivity and Efficiency

Generative AI tools, such as GitHub Copilot and Microsoft Copilot, significantly enhance workplace productivity by automating routine tasks and providing intelligent suggestions. This automation allows employees to focus on more complex and creative aspects of their work, leading to substantial time savings and fewer issues. For instance, GitHub Copilot has been shown to save developers 36% of their time and reduce issues by 48% when working on familiar tasks [Data: Reports (87)].

#### Broad Impact Across Various Roles

The productivity benefits of generative AI tools are particularly pronounced for information workers and roles focused on communication, such as customer service and sales. These tools help in creating and editing more documents, reducing time spent on emails, and improving overall efficiency in communication and document creation [Data: Reports (27, 74, 63,

In [9]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,27,Microsoft and LinkedIn: AI at Work and Product...,0.941176,# Microsoft and LinkedIn: AI at Work and Produ...,9.0
1,74,Copilot and Its Impact on Workplace Productivity,0.823529,# Copilot and Its Impact on Workplace Producti...,9.5
2,71,Impact of Copilot on Workplace Productivity,0.411765,# Impact of Copilot on Workplace Productivity\...,9.0
3,63,Impact of Copilot for Microsoft 365 on Enterpr...,0.235294,# Impact of Copilot for Microsoft 365 on Enter...,9.0
4,85,AI and Productivity Report by Microsoft,0.235294,# AI and Productivity Report by Microsoft\n\nT...,9.0
...,...,...,...,...,...
75,76,Impact of Generative AI on Workplace Productiv...,0.176471,# Impact of Generative AI on Workplace Product...,8.5
76,53,"Tasks, Knowledge, Skills, and Machines in Work...",0.117647,"# Tasks, Knowledge, Skills, and Machines in Wo...",9.0
77,101,Predictive Models and Model Performance in AI,0.117647,# Predictive Models and Model Performance in A...,9.0
78,103,AI and Productivity Research Community: Key Co...,0.117647,# AI and Productivity Research Community: Key ...,9.0


In [10]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 6. LLM tokens: 60150
